<a href="https://colab.research.google.com/github/boffandre-cyber/PEE---2025/blob/main/C%C3%B3pia_de_C%C3%B3pia_de_TRB1_PEE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ainda em falta:

*   ajustar os comentários durante o código (padronizar mais), escrever uma breve
*   introdução explicando objetivos,
*   colocar todas as bibliotecas que estão sendo usadas na primeira secção de código e explica-las no texto,
*   ajustar texto dos gráficos e alguns gráficos com subíndices e explicar cada parte,
*   fazer a parte de texto do reservatório
* Melhorar os atuais textos

Introdução
---
___

Importando Biblitecas e suas funções
---
---
Esta secção de códogo é dedicada e importar todas as bibliotecas que serão utilizadas ao longo do projeto, sendo elas:
*   Pandas:
*   Numpy:
*   Matplotlib:



In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

Lendo o arquivo de dados
---
---
Inicialmente foi buscado um arquivo fluvial no site da ANA (Agencia Nacional das Águas) o qual está no formato csv, portanto deve-se ajusta-lo para a leitura correta pela Biblioteca Pandas, afim de fazer uma análise de dados posterior em forma de dataframe


In [ ]:

# Esta função é necessária para lidar com o formato específico do arquivo,
# que contém metadados antes do cabeçalho.
def read_complex_csv(file_path):
    """
    Lê um arquivo CSV com metadados e retorna um DataFrame.
    """
    with open(file_path, 'r', encoding='latin-1') as f:
        lines = f.readlines()

    # O cabeçalho está na linha 16 (índice 15).
    header_line = lines[15].strip()
    data_lines = lines[16:]

    # Obter os nomes das colunas, removendo aspas duplas.
    columns = [col.strip('"') for col in header_line.split(';')]

    # Processar as linhas de dados, removendo aspas e substituindo vírgulas por pontos.
    data = []
    for line in data_lines:
        if line.strip():  # Ignorar linhas vazias
            row = [re.sub(r'"', '', val).replace(',', '.') for val in line.split(';')]
            data.append(row)

    # Criar o DataFrame a partir dos dados processados.
    df = pd.DataFrame(data, columns=columns)

    # Remover colunas totalmente vazias ou sem nome
    df = df.dropna(axis=1, how='all')
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

    return df

# Substitua '70100000_Vazoes_I.csv' pelo nome do arquivo.
file_path = '70100000_Vazoes_I.csv'
df = read_complex_csv(file_path)




FileNotFoundError: [Errno 2] No such file or directory: '70100000_Vazoes_I.csv'

Análise de Dados
---
___
Análise inicial de dados onde é ajustado os dados de vazões para estar em conformidade com o estipulado como premissa em edital da disciplina. Nela também são ajustadas as colunas que inicialmente são entendidas como variáveis de String, portanto são ajustadas datas na forma de DateTime e vazões como Float

In [ ]:
# -----------------------------------------------------------------------------
# Início da análise
# -----------------------------------------------------------------------------

# Converter a coluna 'Data' para o formato de data
df['Data'] = pd.to_datetime(df['Data'], format='%d/%m/%Y')

# Encontrar o ano mais recente nos dados
most_recent_year = df['Data'].dt.year.max()
print(f"Ano mais recente nos dados: {most_recent_year}")

# Definir o ano de corte para desconsiderar os dois últimos anos
year_cutoff = most_recent_year - 1
print(f"Os dados serão filtrados até o ano de: {year_cutoff - 1}")

# Filtrar o DataFrame para excluir os dois últimos anos
df_filtered = df[df['Data'].dt.year < year_cutoff].copy()

# Identificar as colunas de vazão, excluindo aquelas que contêm a palavra 'Status'
vazao_cols = [col for col in df_filtered.columns if 'Vazao' in col and 'Status' not in col]
print(f"\nColunas de vazão encontradas: {vazao_cols}")

# Desempilhar o DataFrame para obter uma única coluna com todos os valores de vazão
df_melted_filtered = df_filtered.melt(value_vars=vazao_cols, value_name='Vazao')

# Converter a coluna 'Vazao' para tipo numérico e remover valores ausentes
df_melted_filtered['Vazao'] = pd.to_numeric(df_melted_filtered['Vazao'], errors='coerce')
df_melted_filtered.dropna(subset=['Vazao'], inplace=True)

# Calcular a vazão média de longo termo
long_term_avg = df_melted_filtered['Vazao'].mean()

print(f"\nVazão média de longo termo (excluindo os dois últimos anos): {long_term_avg:.2f} m³/s")

# Definir o intervalo de referência e o valor alvo
lower_bound = 30
upper_bound = 60
target_avg = 45 # Ponto central do intervalo

# Verificar se a vazão média está fora do intervalo
if not (lower_bound <= long_term_avg <= upper_bound):
    print(f"\nA vazão média de longo termo está fora do intervalo [{lower_bound}, {upper_bound}] m³/s.")
    print("Aplicando fator de correção.")

    # Calcular o fator de correção
    correction_factor = target_avg / long_term_avg
    print(f"Fator de correção calculado: {correction_factor:.4f}")

    # Criar uma cópia do DataFrame original para aplicar a correção
    df_corrected = df.copy()

    # Aplicar o fator de correção a todas as colunas de vazão
    for col in vazao_cols:
        df_corrected[col] = pd.to_numeric(df_corrected[col], errors='coerce').mul(correction_factor)

    # Recalcular a média de longo termo com os dados corrigidos para verificação
    df_melted_corrected = df_corrected.melt(value_vars=vazao_cols, value_name='Vazao')
    df_melted_corrected['Vazao'] = pd.to_numeric(df_melted_corrected['Vazao'], errors='coerce')
    corrected_avg = df_melted_corrected['Vazao'].mean()
    print(f"Nova vazão média de longo termo após correção: {corrected_avg:.2f} m³/s")

    # Salvar o DataFrame corrigido em um novo arquivo CSV
    df_corrected.to_csv('70100000_Vazoes_I_Corrigido.csv', sep=';', index=False, decimal=',')
    print("\nUm novo arquivo CSV com os dados corrigidos foi gerado e está disponível como '70100000_Vazoes_I_Corrigido.csv'.")
else:
    print(f"\nA vazão média de longo termo já está dentro do intervalo [{lower_bound}, {upper_bound}] m³/s. Nenhuma correção é necessária.")


# Criando uma coluna de mediana:
#df_corrected['Mediana'] = df_corrected[vazao_cols].median(axis=1)

# Ajustantdo as colunas de media minima e maxima do df
df_corrected['Media'] = pd.to_numeric(df_corrected['Media'], errors='coerce')
df_corrected['Minima'] = pd.to_numeric(df_corrected['Minima'], errors='coerce')
df_corrected['Maxima'] = pd.to_numeric(df_corrected['Maxima'], errors='coerce')



for col in ['Media', 'Minima', 'Maxima']:
    df_corrected[col] = df_corrected[col] * correction_factor

# Criando uma coluna de mediana:
df_corrected['Mediana'] = df_corrected[vazao_cols].median(axis=1)

Plotagem dos Gráficos
---
___
Aqui são plotados os gráficos de vazões para o histórico de 40 anos. São feitos os gráficos da vazão média mensal e curva de carga para todo o histórico.
Também nesta secção se encontra a criação de um ano sintético o qual foi utilizado para fazer os gráficos das médias mensais e de Rippl.

In [ ]:
#grafico de vazao da média mensal de todos os anos
df_corrected


#plotando todos os valores de média
df_corrected.plot(x='Data', y='Media', kind='line')
plt.title('Vazão ao longo dos anos')
plt.xlabel('data')
plt.ylabel('vazão média')
plt.grid(True)
plt.show()

#aqui plotar sazonalidades fazer media das medias, das minimas e das maximas dos meses.
df_anosintetico = df_corrected
# Extrai o mês da data
df_anosintetico['Mes'] = df_anosintetico['Data'].dt.month

# Agrupa por mês e calcula a média das médias, minimas, máximas e medianas:
media_mensal_sintetica = df_anosintetico.groupby('Mes')['Media'].mean().reset_index()
media_minima_sintetica = df_anosintetico.groupby('Mes')['Minima'].mean().reset_index()
media_maxima_sintetica = df_anosintetico.groupby('Mes')['Maxima'].mean().reset_index()
mediana_mensal_sintetica = df_anosintetico.groupby('Mes')['Mediana'].mean().reset_index()

# Dicionário para converter número do mês em nome
meses = {
    1: 'Jan', 2: 'Fev', 3: 'Mar', 4: 'Abr', 5: 'Mai', 6: 'Jun',
    7: 'Jul', 8: 'Ago', 9: 'Set', 10: 'Out', 11: 'Nov', 12: 'Dez'
}
media_mensal_sintetica['MesNome'] = media_mensal_sintetica['Mes'].map(meses)
media_minima_sintetica['MesNome'] = media_mensal_sintetica['Mes'].map(meses)
media_maxima_sintetica['MesNome'] = media_mensal_sintetica['Mes'].map(meses)
mediana_mensal_sintetica['MesNome'] = mediana_mensal_sintetica['Mes'].map(meses)

# Ordena corretamente os meses
media_mensal_sintetica = media_mensal_sintetica.sort_values('Mes')


# Junta os três DataFrames em um só, usando 'Mes' como chave
df_plot = media_mensal_sintetica[['Mes', 'Media']].copy()
df_plot['Minima'] = media_minima_sintetica['Minima']
df_plot['Maxima'] = media_maxima_sintetica['Maxima']
df_plot['MesNome'] = df_plot['Mes'].map(meses)

# Ordena corretamente os meses
df_plot = df_plot.sort_values('Mes')

# Plotagem
plt.figure(figsize=(12, 6))

# Faixa entre mínima e máxima
plt.fill_between(
    df_plot['MesNome'],
    df_plot['Minima'],
    df_plot['Maxima'],
    color='lightblue',
    alpha=0.4,
    label='Faixa (Mínima - Máxima)'
)

# Linha da média
plt.plot(df_plot['MesNome'], df_plot['Media'], marker='o', color='blue', label='Média Mensal')

# Linha da mínima
plt.plot(df_plot['MesNome'], df_plot['Minima'], linestyle='--', color='green', label='Média Mínima')

# Linha da máxima
plt.plot(df_plot['MesNome'], df_plot['Maxima'], linestyle='--', color='red', label='Média Máxima')

# Ajustes do gráfico
plt.title('Ano Sintético da Vazão Média Mensal (com Mínima e Máxima)')
plt.xlabel('Mês')
plt.ylabel('Vazão')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

#Plotando a média
plt.figure(figsize=(10, 5))

# Linha da média mensal
plt.plot(media_mensal_sintetica['MesNome'], media_mensal_sintetica['Media'],
         marker='o', color='blue', label='Média Mensal')

# Linha da mediana mensal
plt.plot(
    mediana_mensal_sintetica['MesNome'], mediana_mensal_sintetica['Mediana'],
    marker='s', color='red', linestyle='--', label='Mediana Mensal'
)

# Ajustes do gráfico
plt.title('Ano Sintético - Vazão Média Mensal')
plt.xlabel('Mês')
plt.ylabel('Vazão Média')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


# Plota o gráfico de XXX
df_ordenado = df_corrected
df_ordenado['Media'] = sorted(df_ordenado['Media'])
df_corrected.plot(x='Data', y='Media', kind='line')
plt.title('Curva de XXXX')
plt.xlabel('variable')
plt.ylabel('vazão')
plt.grid(True)
plt.show()


Grafico de Rippl
---
___

In [ ]:


# Plota o gráfico de rippl

# Definir a série de vazões a ser usada
serie_vazao = mediana_mensal_sintetica['Mediana']
vazão_acumulado = serie_vazao.cumsum()
media_medianas = mediana_mensal_sintetica['Mediana'].mean()


# Plotando a figura
plt.figure(figsize=(10, 5))
plt.plot(mediana_mensal_sintetica['MesNome'], vazão_acumulado, color='darkred', linewidth=2)
plt.axhline(0, color='black', linestyle='--')
plt.title('Gráfico de Rippl (Déficit Acumulado)')
plt.xlabel('Tempo')
plt.ylabel('Déficit Acumulado (hm³ ou equivalente)')
plt.grid(True)

# Rippl
Rippl_mensal = serie_vazao - media_medianas
# Plotar gráfico de Rippl
plt.figure(figsize=(10, 5))
plt.plot(media_mensal_sintetica['MesNome'], Rippl_mensal, color='darkred', linewidth=2)
plt.axhline(0, color='black', linestyle='--')
plt.title('Gráfico de Rippl (Déficit Acumulado)')
plt.xlabel('Tempo')
plt.ylabel('Déficit Acumulado (hm³ ou equivalente)')
plt.grid(True)
plt.show()




# Ajustando o gráfico de Rippl para tamanho do reservatório
Rippl_maximo = Rippl_mensal.max()
Rippl_minimo = Rippl_mensal.min()
reservatorio_regularizacao = Rippl_maximo - Rippl_minimo
print(f"O valor máximo de Rippl é: {Rippl_maximo}")
print(f"O valor mínimo de Rippl é: {Rippl_minimo}")
print(f"O valor da regularização do reservatório é: {reservatorio_regularizacao}")


Rippl_corrigido = Rippl_mensal - Rippl_minimo

plt.figure(figsize=(10, 5))
plt.plot(media_mensal_sintetica['MesNome'], Rippl_corrigido, color='darkred', linewidth=2)
plt.axhline(0, color='black', linestyle='--')
plt.title('Gráfico de Rippl (Déficit Acumulado)')
plt.xlabel('Tempo')
plt.ylabel('Déficit Acumulado (hm³ ou equivalente)')
plt.grid(True)

### Reservatório



In [ ]:
# Cálculo do Volume
import math
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection, Line3DCollection

# ----------------- parâmetros -----------------
a = 20.0  # altura máxima (m)

# Entrada do usuário (Enter => usa default)
default_vmax = 1_000_000.0
s = input(f"Informe o volume máximo do reservatório (m³) [padrão {default_vmax:,.0f}]: ").strip()
vmax = float(s) if s else default_vmax
if vmax <= 0:
    raise ValueError("O volume máximo deve ser positivo.")

# ----------------- cálculo de c e dimensões -----------------
# Vmax = 750 * c^2 * a  =>  c = sqrt(Vmax / (750*a))
c = math.sqrt(vmax / (750.0 * a))
w0 = 20.0 * c          # base menor (largura no z=0)
w1 = 40.0 * c          # base maior (largura no z=a)
D  = 25.0 * c          # profundidade (constante)
vmax_calc = 750.0 * (c**2) * a
# ----------------- impressão no console -----------------
print("\n=== RESULTADOS ===")
print(f"a = {a:.2f} m")
print(f"c = {c:.6f} m (fixo a partir de agora)")
print(f"Base menor (20c) = {w0:.3f} m, Base maior (40c) = {w1:.3f} m, Profundidade (25c) = {D:.3f} m")
print(f"Volume máximo informado = {vmax:,.3f} m³")
print(f"Volume máximo calculado (a partir de c) = {vmax_calc:,.3f} m³")

Informe o volume máximo do reservatório (m³) [padrão 1,000,000]: 1000000

=== RESULTADOS ===
a = 20.00 m
c = 8.164966 m (fixo a partir de agora)
Base menor (20c) = 163.299 m, Base maior (40c) = 326.599 m, Profundidade (25c) = 204.124 m
Volume máximo informado = 1,000,000.000 m³
Volume máximo calculado (a partir de c) = 1,000,000.000 m³


In [ ]:
# ============================================================
# Leitura e balanço mensal - Reservatório
# Lê: Mês, Qsintética, Qsaida, Evaporação, Precipitação, [Dias]
# Salva a tabela em: C:\Users\boffa\Downloads\TMP\BalancoReservatorio_.xlsx
# ============================================================

import math, os
import numpy as np
import pandas as pd

# ---- checagem: a e c devem existir (da célula 1) ----
try:
    a, c
except NameError:
    raise RuntimeError("As variáveis 'a' e 'c' não existem no ambiente. Rode as células anteriores primeiro.")

# ---- dimensões fixas e funções geométricas ----
Vmax = 750.0*(c**2)*a
def V_de_b(b):
    return 500.0*(c**2)*b + (250.0*(c**2)/a)*(b**2)

def b_de_V(V):
    V = max(0.0, min(V, Vmax))
    A = 250.0*(c**2)/a
    B = 500.0*(c**2)
    C = -V
    disc = B*B - 4*A*C
    if disc < 0:
        return 0.0
    r1 = (-B + math.sqrt(disc))/(2*A)
    r2 = (-B - math.sqrt(disc))/(2*A)
    b = max(r1, r2, 0.0)
    return max(0.0, min(b, a))

def A_lamina(b):
    return 500.0*(c**2)*(1.0 + b/a)

def _to_float(x):
    if pd.isna(x): return np.nan
    if isinstance(x, (int, float)): return float(x)
    return float(str(x).replace(',', '.'))

def _col(df, nome):
    alvo = nome.strip().lower()
    for k in df.columns:
        if k.strip().lower() == alvo:
            return k
    raise KeyError(f"Coluna obrigatória não encontrada: '{nome}'. Presentes: {list(df.columns)}")

# ---- caminho do arquivo de entrada ----
ARQ = r'Reservatorio_.xlsx'
if not os.path.exists(ARQ):
    raise FileNotFoundError(f"Arquivo não encontrado: {ARQ}")

df = pd.read_excel(ARQ)

# ---- normalização e conversões ----
for nome in ["Qsintética","Qsaida","Evaporação","Precipitação"]:
    df[_col(df, nome)] = df[_col(df, nome)].map(_to_float)

# Dias (se não existir, usa calendário padrão)
dias_padrao = [31,28,31,30,31,30,31,31,30,31,30,31]
if any(cn.strip().lower()=="dias" for cn in df.columns):
    df["Dias"] = df[_col(df, "Dias")].map(_to_float)
else:
    df["Dias"] = pd.Series(dias_padrao[:len(df)])

# ---- balanço mês a mês ----
seg_dia = 86400.0
n = len(df)

Vi_list, Vf_list, bi_list, bf_list = [], [], [], []
Ai_list, Af_list, dV_Q_list, dV_PE_list = [], [], [], []

V_i = 0.0
for i in range(n):
    Qin  = _to_float(df[_col(df,"Qsintética")].iloc[i])
    Qout = _to_float(df[_col(df,"Qsaida")].iloc[i])
    Ev_mm = _to_float(df[_col(df,"Evaporação")].iloc[i])
    Pr_mm = _to_float(df[_col(df,"Precipitação")].iloc[i])
    dias  = float(df["Dias"].iloc[i])

    dV_Q = (max(Qin,0.0) - max(Qout,0.0)) * seg_dia * max(dias,0.0)
    E_m, P_m = 0.001*max(Ev_mm,0.0), 0.001*max(Pr_mm,0.0)

    b_i, A_i = b_de_V(V_i), A_lamina(b_de_V(V_i))

    # iteração p/ área média
    V_guess = V_i + dV_Q + (P_m - E_m) * A_i
    for _ in range(30):
        b_f = b_de_V(V_guess)
        A_f = A_lamina(b_f)
        A_med = 0.5*(A_i + A_f)
        V_new = V_i + dV_Q + (P_m - E_m) * A_med
        if abs(V_new - V_guess) <= 1e-6*max(1.0, Vmax):
            V_guess = V_new
            break
        V_guess = V_new
    V_f = max(0.0, min(V_guess, Vmax))

    b_f, A_f = b_de_V(V_f), A_lamina(b_de_V(V_f))
    dV_PE = (P_m - E_m) * 0.5*(A_i + A_f)

    Vi_list.append(V_i); Vf_list.append(V_f)
    bi_list.append(b_i); bf_list.append(b_f)
    Ai_list.append(A_i); Af_list.append(A_f)
    dV_Q_list.append(dV_Q); dV_PE_list.append(dV_PE)

    V_i = V_f

# ---- tabela final ----
out = pd.DataFrame({
    "Mês": df[_col(df,"Mês")] if "Mês" in df.columns else np.arange(1, n+1),
    "Dias": df["Dias"].astype(int),
    "Q_in (m3/s)": df[_col(df,"Qsintética")].round(3),
    "Q_out (m3/s)": df[_col(df,"Qsaida")].round(3),
    "Evap (mm)": df[_col(df,"Evaporação")].round(1),
    "Prec (mm)": df[_col(df,"Precipitação")].round(1),
    "Vi (m3)": np.array(Vi_list).round(2),
    "Vf (m3)": np.array(Vf_list).round(2),
    "b_i (m)": np.array(bi_list).round(3),
    "b_f (m)": np.array(bf_list).round(3),
    "A_i (m2)": np.array(Ai_list).round(2),
    "A_f (m2)": np.array(Af_list).round(2),
    "ΔV_Q (m3)": np.array(dV_Q_list).round(2),
    "ΔV_P-E (m3)": np.array(dV_PE_list).round(2),
})

# ---- salvamento em Excel ----
out_path = r'BalancoReservatorio_.xlsx'
out.to_excel(out_path, index=False)

out  # mostra a tabela no Jupyter
print(f"\nArquivo Excel salvo em: {out_path}")
print(f"Vmax (m3) = {Vmax:,.2f} | a = {a:.2f} m | c = {c:.4f} m (fixo)")


Arquivo Excel salvo em: BalancoReservatorio_.xlsx
Vmax (m3) = 1,000,000.00 | a = 20.00 m | c = 8.1650 m (fixo)


In [ ]:
# --- COLAB: preparar widgets ---
!pip -q install ipywidgets==8.1.3
from google.colab import output
output.enable_custom_widget_manager()  # habilita exibição de widgets no Colab


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 49.9 MB/s eta 0:00:00


In [ ]:
# ============================================
## Requisitos: 'a', 'c' e DataFrame 'out' com b_f
# ============================================
import io, math, time, threading
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection, Line3DCollection
import ipywidgets as widgets
from IPython.display import display

# ---------------- checagens ----------------
try:
    a, c
except NameError:
    raise RuntimeError("Rode antes a célula que define 'a' e 'c'.")
try:
    out
except NameError:
    raise RuntimeError("Rode antes a célula do balanço mensal que cria 'out'.")

# ---------------- funções geométricas ----------------
w0, w1, D = 20.0*c, 40.0*c, 25.0*c
def largura(z):        return 20.0*c*(1.0 + z/a)
def V_de_b(b):         return 500.0*(c**2)*b + (250.0*(c**2)/a)*(b**2)
def A_lamina(b):       return 500.0*(c**2)*(1.0 + b/a)

# ---------------- trajetória b(t) ----------------
FRAMES_POR_MES = 6  # suavidade 4–8 é bom no Colab
b_knots = []
if   "b_i (m)" in out.columns: b_knots.append(float(out["b_i (m)"].iloc[0]))
elif "b_i"     in out.columns: b_knots.append(float(out["b_i"].iloc[0]))
else:                           b_knots.append(0.0)

if   "b_f (m)" in out.columns: b_mes = out["b_f (m)"].astype(float).values
elif "b_f"     in out.columns: b_mes = out["b_f"].astype(float).values
else: raise RuntimeError("A tabela 'out' precisa ter a coluna 'b_f (m)' (ou 'b_f').")
b_knots.extend(b_mes.tolist())

b_traj = []
for i in range(len(b_knots)-1):
    b0, b1 = b_knots[i], b_knots[i+1]
    for t in np.linspace(0.0, 1.0, FRAMES_POR_MES, endpoint=False):
        b_traj.append((1-t)*b0 + t*b1)
b_traj.append(b_knots[-1])
N = len(b_traj)

# ---------------- geometrias fixas ----------------
bottom = [(0, 0, 0), (w0, 0, 0), (w0, D, 0), (0, D, 0)]
top    = [(0, 0, a), (w1, 0, a), (w1, D, a), (0, D, a)]
edges_res = [
    [bottom[0], bottom[1]], [bottom[1], bottom[2]], [bottom[2], bottom[3]], [bottom[3], bottom[0]],
    [top[0], top[1]], [top[1], top[2]], [top[2], top[3]], [top[3], top[0]],
    [bottom[0], top[0]], [bottom[1], top[1]], [bottom[2], top[2]], [bottom[3], top[3]],
]

def faces_agua_from_b(b):
    wb = largura(b)
    top_b = [(0, 0, b), (wb, 0, b), (wb, D, b), (0, D, b)]
    return [
        bottom, top_b,
        [bottom[0], bottom[1], top_b[1], top_b[0]],
        [bottom[1], bottom[2], top_b[2], top_b[1]],
        [bottom[2], bottom[3], top_b[3], top_b[2]],
        [bottom[3], bottom[0], top_b[0], top_b[3]],
    ]

# ---------------- render de um frame (PNG em memória) ----------------
FIGSIZE = (8.0, 5.0); DPI = 110; VERTICAL_EXAG = 0.65

def render_frame_png(b) -> bytes:
    fig = plt.figure(figsize=FIGSIZE, constrained_layout=True)
    ax  = fig.add_subplot(111, projection='3d')

    poly = Poly3DCollection(faces_agua_from_b(b), alpha=0.65, edgecolor='k')
    poly.set_facecolor((0.2, 0.4, 1.0, 0.65))
    ax.add_collection3d(poly)
    ax.add_collection3d(Line3DCollection(edges_res, linewidths=1.0, colors='k'))

    ax.set_xlim(0, max(w0, w1)); ax.set_ylim(0, D); ax.set_zlim(0, a)
    ax.set_xlabel('Largura (m)'); ax.set_ylabel('Profundidade (m)'); ax.set_zlabel('Altura (m)')
    ax.view_init(elev=22, azim=-60)

    bx, by = max(w0, w1), D
    bz = max(a, VERTICAL_EXAG*(bx+by)/2)
    ax.set_box_aspect((bx, by, bz))

    fig.suptitle(
        f"Nível b = {b:6.2f} m   |   V(b) = {V_de_b(b):,.0f} m³   |   A(b) = {A_lamina(b):,.0f} m²",
        fontsize=11.5, y=0.98
    )

    buf = io.BytesIO()
    fig.savefig(buf, format='png', dpi=DPI, bbox_inches="tight", pad_inches=0.05)
    plt.close(fig)
    return buf.getvalue()

# ---------------- gera frames uma única vez ----------------
frames_png = [render_frame_png(b) for b in b_traj]

# ---------------- widgets ----------------
img      = widgets.Image(value=frames_png[0], format='png')
slider   = widgets.IntSlider(value=0, min=0, max=N-1, step=1,
                             description='Frame', continuous_update=False,
                             layout=widgets.Layout(width='60%'))
btn_prev = widgets.Button(description='⟨⟨', tooltip='Anterior', layout=widgets.Layout(width='60px'))
btn_next = widgets.Button(description='⟩⟩', tooltip='Próximo',  layout=widgets.Layout(width='60px'))

toggle   = widgets.ToggleButton(value=False, description='▶ Play', icon='play',
                                tooltip='Iniciar/Parar', layout=widgets.Layout(width='100px'))
speed    = widgets.IntSlider(value=180, min=60, max=1000, step=20, description='ms/frame',
                             continuous_update=False, layout=widgets.Layout(width='30%'))

def on_slider_change(change):
    if change['name'] == 'value':
        img.value = frames_png[change['new']]
slider.observe(on_slider_change, names='value')

def _prev(_): slider.value = max(slider.min, slider.value-1)
def _next(_): slider.value = slider.min if slider.value >= slider.max else slider.value+1
btn_prev.on_click(_prev); btn_next.on_click(_next)

# ---------------- player com THREAD (estável no Colab) ----------------
play_thread = None
stop_event  = None

def _player_loop():
    while not stop_event.is_set():
        # avança um frame
        if slider.value >= slider.max:
            slider.value = slider.min
        else:
            slider.value += 1
        time.sleep(max(0.01, speed.value/1000))

def on_toggle_change(change):
    global play_thread, stop_event
    if change['name'] != 'value':
        return
    if change['new']:  # iniciar
        toggle.description = '❚❚ Pause'; toggle.icon = 'pause'
        # encerra thread anterior se existir
        if play_thread and play_thread.is_alive():
            stop_event.set(); play_thread.join(timeout=0.2)
        stop_event = threading.Event()
        play_thread = threading.Thread(target=_player_loop, daemon=True)
        play_thread.start()
    else:              # parar
        toggle.description = '▶ Play'; toggle.icon = 'play'
        if play_thread and play_thread.is_alive():
            stop_event.set(); play_thread.join(timeout=0.2)

toggle.observe(on_toggle_change, names='value')

# ---------------- exibir UI ----------------
ui = widgets.HBox([toggle, btn_prev, btn_next, slider, speed])
display(ui, img)


Image(value=b"\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02R\x00\x00\x02'\x08\x06\x00\x00\x00S0\xb7\x01\x00\…